# IK Solvers Tests

In [1]:
import rospy
import moveit_commander
import moveit_msgs.msg
import moveit_msgs.srv
import sys
import csv
import tf_conversions
import geometry_msgs.msg
import golem_right_arm_kinematics as grk
import golem_left_arm_kinematics as glk
import timeit
import math

## Define some functions to work with ROS Services

In [9]:
# Transform a pose tuple into MoveIt! IK request msg
def ik_req_from_pose((x, y, z, yaw, pitch, roll), solver='custom'):
    # srv request definition
    ik_req = moveit_msgs.srv.GetPositionIKRequest()
    
    if solver == 'kdl':
        quaternion = tf_conversions.transformations.quaternion_from_euler(yaw, pitch, roll)
    elif solver == 'custom':
        quaternion = tf_conversions.transformations.quaternion_from_euler(pitch, yaw, roll, axes='szyx')

    # Desired end effector position
    desired_pose = geometry_msgs.msg.PoseStamped()
    desired_pose.pose.position.x = x
    desired_pose.pose.position.y = y
    desired_pose.pose.position.z = z
    desired_pose.pose.orientation.x = quaternion[0]
    desired_pose.pose.orientation.y = quaternion[1]
    desired_pose.pose.orientation.z = quaternion[2]
    desired_pose.pose.orientation.w = quaternion[3]

    ik_req.ik_request.group_name = 'right_arm_kdl'
    ik_req.ik_request.robot_state = robot.get_current_state()
    # ik_req.ik_request.constraints
    ik_req.ik_request.avoid_collisions = True
    ik_req.ik_request.ik_link_name = 'r_eef'
    ik_req.ik_request.pose_stamped = desired_pose
    ik_req.ik_request.timeout.secs = 5
    ik_req.ik_request.attempts = 0
    
    return ik_req

# Transform MoveIt! IK response msg into a list of joints
def joint_list_from_ik_resp(ikresp, joint_names):
    joint_dic = {}
    joint_list = []
    joints = ikresp.solution.joint_state.name
    angles = ikresp.solution.joint_state.position
    
    for i in range(len(joints)):
        joint_dic[joints[i]] = angles[i]
        
    for j in range(len(joint_names)):
        if joint_names[j] in joint_dic.keys():
            joint_list.append(round(joint_dic[joint_names[j]],6))
        
    return joint_list


# Tranform a list of joints into a MoveIt! FK request msg
def fk_req_from_joint_list(joints):
    complete_joints = (0.0, joints[0], 0.0, 0.0, 
                      joints[1], 
                      joints[2],
                      joints[3],
                      joints[4],
                      joints[5],
                      0.0, 0.0, joints[1], joints[2], joints[3], joints[4], joints[5], 0.0, 0.0)
    
    # srv request definition
    fk_req = moveit_msgs.srv.GetPositionFKRequest()
    fk_req.fk_link_names = ['r_eef']
    fk_req.robot_state = robot.get_current_state()
    fk_req.robot_state.joint_state.position = complete_joints
    
    return fk_req

# Transform a MoveIt! FK response msg into a list with the pose
def pose_from_fk_resp(fk_resp):
    msg_pose = fk_resp.pose_stamped[0].pose
    
    rpy = tf_conversions.transformations.euler_from_quaternion((
        msg_pose.orientation.x, 
        msg_pose.orientation.y, 
        msg_pose.orientation.z,
        msg_pose.orientation.w))

    pose = (
        round(msg_pose.position.x,6), 
        round(msg_pose.position.y,6), 
        round(msg_pose.position.z,6), 
        round(rpy[0],6), 
        round(rpy[1],6), 
        round(rpy[2],6)
    )
    
    return pose

# Compute error of IK solutions
# The inputs are the desired pose and the FK of IK solution
def compute_error(desired_pose, solution):
    error = 0.0
    for k in range(len(desired_pose)):
        error += (desired_pose[k] - solution[k])/desired_pose[k]

    return abs(error)

# Transform to quaternion from roll-pitch-yaw
def rpy2quaternion(yaw, pitch, roll):
    t0 = math.cos(roll*0.5)
    t1 = math.sin(roll*0.5)
    t2 = math.cos(yaw*0.5)
    t3 = math.sin(yaw*0.5)
    t4 = math.cos(pitch*0.5)
    t5 = math.sin(pitch*0.5)
    
    q = (
        t0 * t3 * t4 - t1 * t2 * t5,
        t0 * t2 * t5 + t1 * t3 * t4,
        t1 * t2 * t4 - t0 * t3 * t5,
        t0 * t2 * t4 + t1 * t3 * t5
    )
    return q

# Transform a roll-pitch-yaw orientation to a quaternion
def quaternion2rpy(x, y, z, w):
    sp = 2*(x*z - w*y)
    roll = math.atan2(2*(-x*y + z*w), 1 - 2*(y**2 + z**2))
    pitch = math.asin(sp)
    yaw = math.atan2(2*(-x*w + y*z), 1 - 2*(z**2 + w**2))
    
    return (yaw, pitch, roll)

## 1. MoveIt! Setup
**NOTE:** Before running the next code, golem_moveit.launch must be launched!

In [3]:
moveit_commander.roscpp_initialize(sys.argv)

# Robot commander object -> interface the robot as a whole
robot = moveit_commander.RobotCommander()
# MoveGroup object
right_arm = moveit_commander.MoveGroupCommander("right_arm_kdl")

print "============ ROBOT COMMANDER STARTED! ============="

============ ROBOT COMMANDER STARTED! =============


## 2. Import test points from file

In [5]:
# Read file and storage poses in 'test_pose' variable as tuples
f = open('ik_test_poses/left_arm_test_poses(1000).csv', 'r')
test_poses = []
for line in f:
    pose = []
    for string in line.split(','):
        pose.append(round(float(string),6))
    pose = tuple(pose)
    test_poses.append(pose)
test_poses = tuple(test_poses)
print len(test_poses), 'poses read!'

# e = 0
# for pose in test_poses:
#     ikt = glk.inverse_kinematics(*pose)
#     if not ikt:
#         e+=1
# print e, 'poses without solution.'

100 poses read!


In [4]:
test_poses[3]

(-0.196146, -0.411833, 1.313852, 0.175251, -0.851744, -2.388338)

## 3. Tests
### 3.1 Init service clients for FK and IK

In [4]:
# Init service client object
rospy.wait_for_service('/golem_kinematics/left_arm/compute_fk')
rospy.wait_for_service('/golem_kinematics/left_arm/compute_ik')
print "FK and IK services were found!"

compute_custom_ik = rospy.ServiceProxy('/golem_kinematics/left_arm/compute_ik', moveit_msgs.srv.GetPositionIK)
compute_custom_ik_op = rospy.ServiceProxy('/golem_kinematics/left_arm/compute_ik_only_position', moveit_msgs.srv.GetPositionIK)
# compute_kdl_ik = rospy.ServiceProxy('/compute_ik', moveit_msgs.srv.GetPositionIK)
compute_custom_fk = rospy.ServiceProxy('/golem_kinematics/left_arm/compute_fk', moveit_msgs.srv.GetPositionFK)
# compute_kdl_fk = rospy.ServiceProxy('/compute_fk', moveit_msgs.srv.GetPositionFK)
print "Service client objects created!"

FK and IK services were found!
Service client objects created!


In [10]:
test_pose = (1.0, 0.0, 1.3, 0.0, 0.0, 0.0)

print '-------------------------------'
print '---------- TEST ', 1, ' ----------'
print '-------------------------------'

# Transform pose tuple to IK MoveIt! msg
cs_ik_req = ik_req_from_pose(test_pose,'custom')

# IK computation
cs_ik = compute_custom_ik_op(cs_ik_req)

# Transform IK response into a joint list
print '### Inverse Kinematics ###'
cs_sol = joint_list_from_ik_resp(cs_ik, ('m0', 'm20', 'm21', 'm22', 'm23', 'm24'))
print 'Custom Solver:  ', cs_sol

# Transform joint list to FK request and FK computation
print '\n### Forward Kinematics ###'
print 'Pose:           ', test_pose
if cs_ik.solution.joint_state.position:
    cs_fk_req = fk_req_from_joint_list(cs_sol)
    cs_fk_resp = compute_custom_fk(cs_fk_req)
    cs_fk = pose_from_fk_resp(cs_fk_resp)
    print 'Custom Solver:  ', cs_fk

-------------------------------
---------- TEST  1  ----------
-------------------------------
### Inverse Kinematics ###
Custom Solver:   [-0.837832, -1.12992, 1.967752, 0.625803, -0.065803, 0.0]

### Forward Kinematics ###
Pose:            (1.0, 0.0, 1.3, 0.0, 0.0, 0.0)
Custom Solver:   (1.0, -0.0, 1.3, 0.0, -0.56, 0.0)


In [8]:
# Make an experiment for each given pose.
csvfile = open('test_results_ik001.csv', 'wb')
writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
for pose in range(len(test_poses)):
    print '-------------------------------'
    print '---------- TEST ', pose+1, ' ----------'
    print '-------------------------------'
    
    # Transform pose tuple to IK MoveIt! msg
    cs_ik_req = ik_req_from_pose(test_poses[pose],'custom')
    # kdl_ik_req = ik_req_from_pose(test_poses[pose],'kdl')

    # IK computation
    t0 = timeit.default_timer() 
    cs_ik = compute_custom_ik(cs_ik_req)
    t1 = timeit.default_timer() 
    
    # t2 = timeit.default_timer() 
    # kdl_ik = compute_kdl_ik(kdl_ik_req)
    # t3 = timeit.default_timer()
    
    cs_time = t1 - t0
    # kdl_time = t3 - t2
    
    # Transform IK response into a joint list
    print '### Inverse Kinematics ###'
    cs_sol = joint_list_from_ik_resp(cs_ik, ('m0', 'm20', 'm21', 'm22', 'm23', 'm24'))
    # kdl_sol = joint_list_from_ik_resp(kdl_ik, ('m0', 'm10', 'm11', 'm12', 'm13', 'm14'))
    print 'Custom Solver:  ', cs_sol
    # print 'KDL Solver:     ', kdl_sol
    
    # Transform joint list to FK request and FK computation
    print '\n### Forward Kinematics ###'
    print 'Pose:           ', test_poses[pose]
    if cs_ik.solution.joint_state.position:
        cs_fk_req = fk_req_from_joint_list(cs_sol)
        cs_fk_resp = compute_custom_fk(cs_fk_req)
        cs_fk = pose_from_fk_resp(cs_fk_resp)
        print 'Custom Solver:  ', cs_fk

    # if kdl_ik.solution.joint_state.position:
    #     kdl_fk_req = fk_req_from_joint_list(kdl_sol)
    #     kdl_fk_resp = compute_custom_fk(kdl_fk_req)
    #     kdl_fk = pose_from_fk_resp(kdl_fk_resp)
        # print 'KDL Solver:     ', kdl_fk
       
    # Error computation
    print '\n### Errors ###'
    
    cs_error = []
    for i in range(3):
        if cs_ik.solution.joint_state.position:
            cs_error.append(abs(cs_fk[i] - test_poses[pose][i])/test_poses[pose][i])
        else:
            cs_error.append('no')
    print 'Custom Solver:  ', cs_error
        
    # kdl_error = []
    # for j in range(6):
    #     if kdl_ik.solution.joint_state.position:
    #         kdl_error.append(abs(kdl_fk[j] - test_poses[pose][j])/test_poses[pose][i])
    #     else:
    #        kdl_error.append(-1)
    # print 'KDL Solver:     ', kdl_error
        
    data = cs_error
    data.append(cs_time)
    # data += kdl_error
    # data.append(kdl_time)
    writer.writerow(data)
        
    print '\n'
    
csvfile.close()

-------------------------------
---------- TEST  1  ----------
-------------------------------
### Inverse Kinematics ###
Custom Solver:   [0.750945, 1.449907, -1.132452, 0.278868, -1.240424, -1.328203]

### Forward Kinematics ###
Pose:            (0.064694, 0.251447, 0.712646, -1.328203, 0.961556, 1.0684)
Custom Solver:   (0.064694, 0.251447, 0.712646, -1.328203, 0.961556, 1.0684)

### Errors ###
Custom Solver:   [0.0, 0.0, 0.0]


-------------------------------
---------- TEST  2  ----------
-------------------------------
### Inverse Kinematics ###
Custom Solver:   [0.436943, -0.398842, 1.931098, 1.471565, -1.594355, 0.749859]

### Forward Kinematics ###
Pose:            (-0.148739, 0.918744, 1.2353, 0.749859, 0.12279, 1.9692)
Custom Solver:   (-0.148738, 0.918744, 1.2353, 0.749859, 0.12279, 1.9692)

### Errors ###
Custom Solver:   [-6.723186252435474e-06, 0.0, 0.0]


-------------------------------
---------- TEST  3  ----------
-------------------------------
### Inverse Kinematic